In [35]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
import sys
import time
import numpy as np
import json
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt 
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import (NNConv, GMMConv, GraphConv, Set2Set)
from torch_geometric.nn import (SplineConv, graclus, max_pool, max_pool_x, global_mean_pool)

#from neuralnet_pytorch.metrics import chamfer_loss

import trimesh

from visualization_utils import plot_mesh_3d

from models import *
from datasets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
def train(epoch, model, train_loader, device, optimizer):
    model.train()

    for data in tqdm(train_loader, leave=False):
        data = data.to(device)
        optimizer.zero_grad()
        verticies_preds, global_preds = model(data)
        
        vert_loss = F.mse_loss(verticies_preds, data.y)
        global_loss = F.mse_loss(global_preds[:, :3], data.pressure_drag.reshape(-1, 3)) +\
                      F.mse_loss(global_preds[:, 3:6], data.viscous_drag.reshape(-1, 3)) +\
                      F.mse_loss(global_preds[:, 6:9], data.pressure_moment.reshape(-1, 3)) +\
                      F.mse_loss(global_preds[:, 9:], data.viscous_moment.reshape(-1, 3))

        loss = (vert_loss + global_loss) / 2
        
        loss.backward()
        optimizer.step()

        #print('Train Loss: {:.4f}'.format(loss))
    del data, loss, responce

def validate(model, test_loader, device):
    model.eval()
    loss = 0

    for data in tqdm(test_loader):
        data = data.to(device)
        
        verticies_preds, global_preds = model(data)

        vert_loss = F.mse_loss(verticies_preds, data.y)
        global_loss = F.mse_loss(global_preds[:, :3], data.pressure_drag.reshape(-1, 3)) +\
                      F.mse_loss(global_preds[:, 3:6], data.viscous_drag.reshape(-1, 3)) +\
                      F.mse_loss(global_preds[:, 6:9], data.pressure_moment.reshape(-1, 3)) +\
                      F.mse_loss(global_preds[:, 9:], data.viscous_moment.reshape(-1, 3))

        curr_loss = (vert_loss + global_loss) / 2
        
        loss += curr_loss.cpu().detach().numpy()
    return loss / len(test_loader)

def process_model(network, out_file_name, train_loader, validation_loader,
                  init_lr=0.1, num_epochs=150):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = network(train_loader.dataset.num_features).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=init_lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.8, patience=3, min_lr=0.00001, verbose=True)
   
    start_time = time.time()
    print('Start training')
    for epoch in tqdm(range(num_epochs)):
        train(epoch, model, train_loader, device, optimizer)
        test_acc = validate(model, validation_loader, device)
        scheduler.step(test_acc)
        with open("Expirements/" + out_file_name, 'a') as file:
            print('Epoch: {:02d}, Time: {:.4f}, Validation Accuracy: {:.4f}'\
                  .format(epoch, time.time() - start_time, test_acc), file=file)
            
        torch.save(model.state_dict(), "Expirements/" + out_file_name + ".nn")

#         start_time = time.time()
#         test_acc = validate(model, test_loader, device)
#         print('Test, Time: {:.4f}, Accuracy: {:.4f}'\
#               .format(time.time() - start_time, test_acc))

    return model

In [31]:
train_dataset = CDFDataset('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld', 
                           connectivity=10, data_step=10) #, delimetr=0.002
val_dataset   = CDFDataset('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld',
                           connectivity=10, train=False, data_step=10) #, delimetr=0.999

In [32]:
train_loader = torch_geometric.data.DataLoader(train_dataset, batch_size=2, shuffle=False)
val_loader = torch_geometric.data.DataLoader(val_dataset, batch_size=2, shuffle=False)

In [33]:
print(len(train_loader), len(val_loader))

243 27


In [36]:
!pwd

/cvlabdata2/home/artem/DeepSDF


In [34]:
model = process_model(SplineCNN4Pooling, 'Spline4CDFPooling', train_loader, val_loader, init_lr=0.02)

Start training


KeyboardInterrupt: 

In [ ]:
len(train_dataset) + len(val_dataset)

# Compute Statistics for Normalization

In [17]:
objects = list()
for (dirpath, dirnames, filenames) in os.walk('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld'):
    objects += [os.path.join(dirpath, file) for file in filenames if file[-4:] == '.fld']

mean_values = np.zeros((len(objects), 4))
std_values = np.zeros((len(objects), 4))

for idx, fld_path in tqdm(enumerate(objects), total=len(objects)):
    fld = np.genfromtxt(fld_path, delimiter=',', skip_header=1)
    fld[fld > 10e5] = np.nan
    fld = fld[~np.isnan(fld).any(axis=1)]

    answers = fld[:, 3:]

    for f in range(answers.shape[1]):
        mean_values[idx, f] = np.mean(answers[:, f])
        std_values[idx, f] = np.std(answers[:, f])
        
print(np.mean(mean_values, axis=0))
print(np.mean(std_values, axis=0))


[-1.90357614e+00  9.55119907e-02  2.05472217e+02  5.53618263e-05]
[3.71674873e+00 4.93675056e-02 1.10871494e+02 2.63155496e-05]


In [28]:
objects = list()
for (dirpath, dirnames, filenames) in os.walk('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/scr'):
    objects += [os.path.join(dirpath, file) for file in filenames if file[-5:] == '.json' and file[0] != '.']

scr_data_store = np.zeros((len(objects), 12))

for idx, scr_path in tqdm(enumerate(objects), total=len(objects)):
    with open(scr_path) as scr_file:
        scr_data = json.load(scr_file)
        scr_data_store[idx, :3] = scr_data['pressure_drag']
        scr_data_store[idx, 3:6] = scr_data['viscous_drag']
        scr_data_store[idx, 6:9] = scr_data['pressure_moment']
        scr_data_store[idx, 9:] = scr_data['viscous_moment']  
        
print(np.mean(scr_data_store, axis=0))
print(np.std(scr_data_store, axis=0))


[ 7.15702765e-01  9.76291022e-03 -1.97037022e-04  4.33680848e-02
  2.71446501e-03  2.42610894e-05 -1.63100377e-05 -1.20658604e-03
  2.01998814e-01 -2.94244062e-06  1.35224581e-05 -6.22179022e-04]
[3.12011511e-01 2.76790047e-01 4.93472812e-02 7.02184919e-03
 1.78783928e-03 8.31190054e-04 4.32590171e-03 1.71780821e-02
 1.01220579e-01 1.13513395e-04 2.45400068e-04 1.05765967e-03]


# Visualizintion

In [ ]:
from visualization_utils import saveMeshPly

data_instance_10 = make_data_instance_from_fld(
                    '/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/0010_0005.fld')

# data_instance_0 = make_data_instance_from_fld(
#                     '/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/0000_0005.fld')

In [ ]:
saveMeshPly(data_instance_10, 'Expirements/Visualizations/0010gt.ply')
saveMeshPly(data_instance_0, 'Expirements/Visualizations/0000gt.ply')

In [ ]:
data_instance_10_ans = data_instance_10.clone()

In [ ]:
model = SplineCNN4(3)
model.load_state_dict(torch.load("Expirements/Spline4CDF.nn"))

device = "cuda:0"
model.to(device)
data_instance_10 = data_instance_10.to(device)
# data_instance_0 = data_instance_0.to(device)

In [ ]:
data_instance_10_ans.y = model(data_instance_10).cpu().detach()
# ans2 = model(data_instance_0)

In [ ]:
saveMeshPly(data_instance_10_ans, 'Expirements/Visualizations/0010pr.ply')

In [ ]:
sum(p.numel() for p in model.parameters())

# Old Vis

In [ ]:
colors=np.copy(data_instance.y[:, 0])
threshold = np.percentile(colors, 99.5)
colors[colors > threshold] = threshold

plt.hist(colors, bins=100)
plt.show()

In [ ]:
xadj = np.loadtxt('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/xadj/0000.xadj')
fld = np.genfromtxt('/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/0000_0005.fld', delimiter=',', skip_header=1)

fld_coords = sorted(fld[:, :3],key=lambda x: x[0])
xadj_coords = sorted(xadj[:, :3], key=lambda x: x[0])

In [ ]:
xadj.shape

In [ ]:
fld.shape

In [ ]:
xadj 

In [ ]:
from pyntcloud import PyntCloud
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def plot_points_from_fld(fld, step_pt=2):
    colors=np.copy(fld[:,3])
    threshold = np.percentile(colors, 99.5)
    colors[colors > threshold] = threshold
    norm = mpl.colors.Normalize(vmin=np.min(colors), vmax=np.max(colors))
    cmap = cm.hot
    m = cm.ScalarMappable(norm=norm, cmap=cmap)

    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(-fld[::step_pt,2], fld[::step_pt,0], fld[::step_pt,1], s=5, c=m.to_rgba(colors[::step_pt]))
    plt.colorbar(m)
    ax.set_xlabel('X Label')
    ax.set_ylabel('Y Label')
    ax.set_zlabel('Z Label')
    plt.show()

In [ ]:
plot_points_from_fld(fld)

In [ ]:
colors=np.copy(fld[:,3])
threshold = np.percentile(colors, 99.5)
colors[colors > threshold] = threshold

plt.hist(colors, bins=100)
plt.show()

In [ ]:
fld.shape

In [ ]:
from custom_utils import plot_points_from_torch

In [ ]:
plot_points_from_torch(np.array(sorted(fld, key=lambda x: x[0]))[:, :3])

In [ ]:
plot_points_from_torch(np.array(sorted(xadj, key=lambda x: x[0]))[:, :3])

In [ ]:
objects = list()
for (dirpath, dirnames, filenames) in os.walk("/cvlabsrc1/cvlab/dataset_shapenet/code/foam_npy/fld/"):
    objects += [os.path.join(dirpath, file) for file in filenames if file[-4:] == '.fld']

for path in objects:
    a = np.genfromtxt(path, delimiter=',', skip_header=1)
    if a.max() > 1e10: 
        print(a.max(), path)

In [ ]:
a.argmax(axis=0)

In [ ]:
a[80938]